In [1]:
from IPython.display import Image
Image(url='./imgs/gnn_layers.png', width=500)

### two layers GCN

In [2]:
Image(url='./imgs/two-layers-gcn.png', width=400)

## interaction_network

- references
    - https://arxiv.org/pdf/1612.00222.pdf：Learning about Objects, Relations and Physics
        - 网路结构非常清晰；
    - https://github.com/higgsfield/interaction_network_pytorch

### Example

- https://blog.zakjost.com/post/gcn_citeseer/